In [5]:
import pandas as pd

# Load the training and testing data
train_data = pd.read_csv('train.csv/train.csv')
test_data = pd.read_csv('test.csv/test.csv')

# Display the first few rows of the training data
print(train_data.head())
# Display the first few rows of the test data
print(test_data.head())

# Display summary statistics of the training data
print(train_data.describe())
# Display information about the training data
print(train_data.info())


   id    brand          model  model_year  milage fuel_type   
0   0     Ford   F-150 Lariat        2018   74349  Gasoline  \
1   1      BMW          335 i        2007   80000  Gasoline   
2   2   Jaguar      XF Luxury        2009   91491  Gasoline   
3   3      BMW   X7 xDrive40i        2022    2437    Hybrid   
4   4  Pontiac  Firebird Base        2001  111000  Gasoline   

                                              engine   
0      375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel  \
1  300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...   
2       300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel   
3  335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...   
4      200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel   

                     transmission ext_col int_col       accident clean_title   
0                    10-Speed A/T    Blue    Gray  None reported         Yes  \
1                     6-Speed M/T   Black   Black  None reported         Yes   
2                     6-Speed A/T  Purple   

In [8]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import numpy as np

# Separate features and target variable from training data
X = train_data.drop(['id', 'price'], axis=1)
y = train_data['price']

# Select categorical and numerical columns
categorical_cols = [cname for cname in X.columns if X[cname].dtype == "object"]
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = xgb.XGBRegressor(random_state=0)

# Create a pipeline that preprocesses the data and then fits the model
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__max_depth': [3, 5, 7],
    'model__subsample': [0.6, 0.8, 1.0],
    'model__colsample_bytree': [0.6, 0.8, 1.0]
}

# Set up the grid search
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Split data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the final model with best parameters
# Note: We use ColumnTransformer to preprocess the data before fitting the model
best_model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('model', xgb.XGBRegressor(**{k.split('__')[1]: v for k, v in best_params.items()}))])

best_model_pipeline.fit(X_train, y_train)

# Evaluate the model
preds = best_model_pipeline.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, preds))
print('Root Mean Squared Error with best model:', rmse)


Fitting 3 folds for each of 243 candidates, totalling 729 fits
Best parameters found:  {'model__colsample_bytree': 0.6, 'model__learning_rate': 0.01, 'model__max_depth': 7, 'model__n_estimators': 200, 'model__subsample': 0.6}
Root Mean Squared Error with best model: 61735.105681607434


In [9]:
# Make predictions on the test data
test_preds = best_model_pipeline.predict(test_data.drop(['id'], axis=1))

# Create a DataFrame with the ID and predicted price
submission = pd.DataFrame({'id': test_data['id'], 'price': test_preds})

# Save the submission DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)


In [ ]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import numpy as np

# Separate features and target variable from training data
X = train_data.drop(['id', 'price'], axis=1)
y = train_data['price']

# Select categorical and numerical columns
categorical_cols = [cname for cname in X.columns if X[cname].dtype == "object"]
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the base models
base_models = [
    ('xgb', xgb.XGBRegressor(random_state=0)),
    ('rf', RandomForestRegressor(random_state=0))
]

# Define the meta-model
meta_model = RidgeCV()

# Create the stacking model
stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model, n_jobs=-1)

# Bundle preprocessing and stacking model in a pipeline
stacking_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                    ('stacking', stacking_model)])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'stacking__xgb__n_estimators': [100, 200],
    'stacking__xgb__learning_rate': [0.01, 0.05],
    'stacking__xgb__max_depth': [3, 5],
    'stacking__xgb__subsample': [0.6, 0.8],
    'stacking__xgb__colsample_bytree': [0.6, 0.8],
    'stacking__rf__n_estimators': [100, 200],
    'stacking__rf__max_depth': [None, 10]
}

# Set up the grid search
grid_search = GridSearchCV(estimator=stacking_pipeline, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Split data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the final stacking model with best parameters
best_stacking_model = grid_search.best_estimator_

# Evaluate the model
preds = best_stacking_model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, preds))
print('Root Mean Squared Error with best stacking model:', rmse)

# Make predictions on the test data
test_preds = best_stacking_model.predict(test_data.drop(['id'], axis=1))

# Create a DataFrame with the ID and predicted price
submission = pd.DataFrame({'id': test_data['id'], 'price': test_preds})

# Save the submission DataFrame to a CSV file
submission.to_csv('xgboost_with_stacking.csv', index=False)


Fitting 3 folds for each of 128 candidates, totalling 384 fits


In [10]:
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter distributions for RandomizedSearchCV
param_dist = {
    'stacking__xgb__n_estimators': [100, 200],
    'stacking__xgb__learning_rate': [0.01, 0.05],
    'stacking__xgb__max_depth': [3, 5],
    'stacking__xgb__subsample': [0.6, 0.8],
    'stacking__xgb__colsample_bytree': [0.6, 0.8],
    'stacking__rf__n_estimators': [100, 200],
    'stacking__rf__max_depth': [None, 10]
}

# Set up the randomized search
random_search = RandomizedSearchCV(estimator=stacking_pipeline, param_distributions=param_dist, 
                                   n_iter=50, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error', random_state=0)

# Fit the randomized search
random_search.fit(X_train, y_train)

# Get the best parameters
best_params = random_search.best_params_
print("Best parameters found: ", best_params)

# Train the final stacking model with best parameters
best_stacking_model = random_search.best_estimator_

# Evaluate the model
preds = best_stacking_model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, preds))
print('Root Mean Squared Error with best stacking model:', rmse)

# Make predictions on the test data
test_preds = best_stacking_model.predict(test_data.drop(['id'], axis=1))

# Create a DataFrame with the ID and predicted price
submission = pd.DataFrame({'id': test_data['id'], 'price': test_preds})

# Save the submission DataFrame to a CSV file
submission.to_csv('Randomized_search_cv.csv', index=False)


NameError: name 'stacking_pipeline' is not defined